In [1]:
%matplotlib inline

from pathlib import Path

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import math

from keras.models import Sequential 
from keras.layers import Dense 
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras.utils.np_utils import to_categorical
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler

Using TensorFlow backend.


In [2]:
DATA_DIR = Path('.', 'data', 'final', 'public')
train_values = pd.read_csv(DATA_DIR / 'train_values.csv', index_col='building_id')
train_labels = pd.read_csv(DATA_DIR / 'train_labels.csv', index_col='building_id')
test_values = pd.read_csv(DATA_DIR / 'test_values.csv', index_col='building_id')

In [3]:
def inicializamodelo(caso):
    global legeo,lencoder,lencoder_col
    if 'legeo' in globals():
        del legeo
    if 'lencoder' in globals():    
        del lencoder
    if 'lencoder_col' in globals():
        del lencoder_col
    lencoder_col=list([])
    if caso == 1:
        legeo = LabelEncoder()
        lencoder= list([])
        print("Inicializa LabelEncoder caso 1")
    if caso == 2:
        legeo = LabelEncoder()
        lencoder= OneHotEncoder(handle_unknown='ignore', sparse=False)   
        print("Inicializa OneHotEncoder caso 2")

In [4]:
def prepros(ltrain,caso):
    AGELIMIT=100
    global legeo,lencoder,lencoder_col
    s = (ltrain.dtypes == 'object')
    object_cols = list(s[s].index)
    s = (ltrain.dtypes != 'object')
    number_cols = list(s[s].index)
    if caso == 1:
            inicializamodelo(caso)
            contador=0
            lsns_data=ltrain[number_cols].copy()
            for col in ltrain[object_cols].columns:
                lencoder.append(LabelEncoder())
                lencoder[contador].fit(ltrain[col])
                lsns_data[col]=lencoder[contador].transform(ltrain[col])  
                contador=contador+1
    if caso == -1:
            contador=0
            lsns_data=ltrain[number_cols].copy()
            for col in ltrain[object_cols].columns:
                lsns_data[col]=lencoder[contador].transform(ltrain[col])  
                contador=contador+1               
    if caso == 2:  
            inicializamodelo(caso)
            nada = lencoder.fit_transform(ltrain[object_cols])
            co1c=0
            for co1 in lencoder.categories_:
                for co2 in co1:
                    lencoder_col.append(object_cols[co1c]+"_"+co2)
                co1c=co1c+1
            lsns_data=pd.concat([ltrain[number_cols].copy(),
                                 pd.DataFrame(nada,columns=lencoder_col,
                                         index=ltrain[object_cols].index.tolist())],axis=1)
    if caso == -2:  
            lsns_data=ltrain[number_cols].copy()
            nada = lencoder.transform(ltrain[object_cols])
            lsns_data=pd.concat([ltrain[number_cols].copy(),
                                 pd.DataFrame(nada,columns=lencoder_col,
                                         index=ltrain[object_cols].index.tolist())],axis=1)        
    
    geo_level_1_fact=math.pow(10,int(math.log(lsns_data['geo_level_2_id'].max(),10)+1))
    geo_level_2_fact=math.pow(10,int(math.log(lsns_data['geo_level_3_id'].max(),10)+1))
    lsns_data['geo_level_n']=  lsns_data['geo_level_1_id']*geo_level_1_fact*geo_level_2_fact+lsns_data['geo_level_2_id']*geo_level_2_fact+lsns_data['geo_level_3_id']
    lsns_data['geo_level']=lsns_data['geo_level_n'].astype(np.int64).astype(str)
    legeo.fit(lsns_data['geo_level'])
    lsns_data['geo_level_cod']=legeo.transform(lsns_data['geo_level'])
    lsns_data=lsns_data.drop(['geo_level_1_id','geo_level_2_id','geo_level_3_id','geo_level_n','geo_level'],axis=1)
    min_max_scaler = MinMaxScaler()
    lsns_data[list(lsns_data.columns)]=min_max_scaler.fit_transform(lsns_data.values)
    lcolum_x = list(lsns_data.columns)
    return(lsns_data,lcolum_x)   

In [5]:
sns_data,colum_x=prepros(train_values,1)


Inicializa LabelEncoder caso 1


/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [6]:
model = LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial',max_iter=500)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(train_values, train_labels, test_size=0.33, random_state=42)

In [8]:
micaso=2
pred_data,colum_x=prepros(X_train,micaso)
model.fit(pred_data,y_train)
lcolum_f = list(X_train.columns)
test_data,column_t=prepros(X_test,-micaso)
f1=f1_score(y_test, model.predict(test_data), average='micro')  
print('F1_score:',f1)

Inicializa OneHotEncoder caso 2


/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


F1_score: 0.586995197618577


In [9]:
#pred_data,colum_x=prepros(train_values,1)
#model.fit(pred_data,train_labels)
#lcolum_f = list(test_values.columns)
#pvalues=test_values.copy()
#pvalues['damage_grade']=model.predict(pred_data)
#pvalues=pvalues.drop(lcolum_f,axis=1)
#pvalues.to_csv(DATA_DIR / 'submission_00_01.csv')

Inicializa LabelEncoder caso 1


/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ValueError: Length of values does not match length of index